In [ ]:
import plotly.express as px
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from itertools import chain

In [ ]:
import plotly.io as pio

pio.templates["custom_dark_green"] = go.layout.Template(
    layout=dict(
        paper_bgcolor='black',
        plot_bgcolor='black',
        font=dict(color='lightgreen'),
        xaxis=dict(color='lightgreen', gridcolor='gray'),
        yaxis=dict(color='lightgreen', gridcolor='gray')
    )
)

pio.templates.default = "custom_dark_green"

In [ ]:
df = pd.read_csv('Worldwide_Travel_Cities_Dataset.csv')

In [ ]:
df.head()

In [ ]:
theme_cols = ['culture','adventure','nature','beaches','nightlife','cuisine','wellness','urban','seclusion']
budget_map = {'Budget': 3, 'Mid-range': 2, 'Luxury': 1}

def climate_comfort(json_str):
    try:
        temps = json.loads(json_str)
        avg_temps = [temps[str(m)]['avg'] for m in range(1, 13)]
        return -np.mean([abs(t - 23) for t in avg_temps])  # closer to 23°C is better
    except:
        return np.nan

In [ ]:
df['theme_score'] = df[theme_cols].mean(axis=1)
df['budget_score'] = df['budget_level'].map(budget_map)
df['duration_score'] = df['ideal_durations'].apply(len)
df['climate_score'] = df['avg_temp_monthly'].apply(climate_comfort)

In [ ]:
score_cols = ['theme_score', 'budget_score', 'duration_score', 'climate_score']
df_clean = df.dropna(subset=score_cols).copy()

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_clean[score_cols])

In [ ]:
print(X_scaled)

In [ ]:
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# --- PCA Setup (Assuming you already have df and score_cols) ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[score_cols])

pca = PCA(n_components=1)
pca.fit(X_scaled)

weights = pca.components_[0]
feature_weights = dict(zip(score_cols, weights))

# --- Plotly Table ---
feature_names = list(feature_weights.keys())
weight_values = [round(w, 4) for w in feature_weights.values()]

fig = go.Figure(data=[go.Table(
    header=dict(values=["🧮 Feature", "📊 PCA Weight"],
                fill_color='darkslategray',
                font=dict(color='white', size=13),
                align='left'),
    cells=dict(values=[feature_names, weight_values],
               fill_color='#1f77b4',
               align='left',
               font=dict(size=12))
)])

fig.update_layout(title='📊 PCA-Derived Feature Weights', height=400)
fig.show(renderer='iframe')

In [ ]:
df_clean['composite_score'] = sum(df_clean[feature] * weight for feature, weight in feature_weights.items())

In [ ]:
top_cities_df = df_clean.sort_values(by='composite_score', ascending=False).head(8)
top_cities_df_display = top_cities_df[['city', 'country', 'region', 'budget_level']]
top_cities_df_display.reset_index(drop=True, inplace=True)

fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=["🏙️ City", "🌍 Country", "📌 Region", "💸 Budget Level"],
                fill_color="red",
                align="left",
                font=dict(color="white", size=14),
                height=35
            ),
            cells=dict(
                values=[
                    top_cities_df_display['city'],
                    top_cities_df_display['country'],
                    top_cities_df_display['region'],
                    top_cities_df_display['budget_level'],
                ],
                fill_color="#1f77b4",
                align="left",
                font=dict(size=13),
                height=30
            )
        )
    ]
)

fig.update_layout(title="🏆 Top 8 Cities Based on PCA Composite Score", title_font_size=20)
fig.show(renderer='iframe')